In [ ]:
import torch
import json
import openai
from llmcam.fn_to_fc import complete, tool_schema

In [3]:
def detect_objects(image_path: str, conf: float=0.05):
    """Run YOLO object detection on an input image."""
    model = torch.hub.load("ultralytics/yolov5", "yolov5s")
    model.conf = conf  # NMS confidence threshold
    
    results = model(image_path)
    count = results.pandas().xyxy[0]['name'].value_counts().to_dict()
    return json.dumps(count, indent=2)

In [ ]:
messages = [
    {
        "role":"system", 
        "content":"You are a helpful system administrator. Use the supplied tools to assist the user. Only use one tool at a time.\
        If you cannot decide which tool to use, ask the user for more information."    
    },
]
# tools = [tool_schema(detect_objects)]
# def fn_name(res): return res.tool_calls[0].function.name
# def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
# def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res)) # This one cannot be imported for use from other notebooks, hence the copy
# def fn_result_content(res):
#     """Create a content containing the result of the function call"""
#     content = dict()
#     content.update(fn_args(res))
#     content.update({fn_name(res): fn_exec(res)})
#     return json.dumps(content)
# # generate_messages will use the tools defined in 06_fn_to_fc.ipynb
# def complete(
#     role: str,  # The role of the message sender; Literal is not supported on my computer
#     content: str,  # The content of the message
#     tool_call_id=None):
#     """Send completion request with messages, and save the response in messages again"""
#     messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
#     response = openai.chat.completions.create(
#         model="gpt-4o", 
#         messages=messages, 
#         tools=tools
#     )
#     res = response.choices[0].message
#     messages.append(res.to_dict())
#     if res.to_dict().get('tool_calls'):
#         complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)
#     return messages[-1]['role'], messages[-1]['content']

In [5]:
messages = [
    {
        "role":"system", 
        "content":"You are a helpful system administrator. Use the supplied tools to assist the user. Only use one tool at a time.\
        If you cannot decide which tool to use, ask the user for more information."    
    },
]
tools = [tool_schema(detect_objects)]
complete(messages, 'user', 'Can you detect the objects in this image\
          https://homebodyeats.com/wp-content/uploads/2021/11/thanksgiving-cheese-board.jpg? with threshold 0.1', tools)

TypeError: 'NoneType' object is not callable

In [5]:
complete("user", "Can you detect the objects in this image https://homebodyeats.com/wp-content/uploads/2021/11/thanksgiving-cheese-board.jpg? with threshold 0.1")
for message in messages:
    print(f"{message['role'].capitalize()}: {message['content']}")

Using cache found in /home/abc/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-10 Python-3.10.15 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/home/abc/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


System: You are a helpful system administrator. Use the supplied tools to assist the user. Only use one tool at a time.        If you cannot decide which tool to use, ask the user for more information.
User: Can you detect the objects in this image https://homebodyeats.com/wp-content/uploads/2021/11/thanksgiving-cheese-board.jpg? with threshold 0.1
Assistant: None
Tool: {"image_path": "https://homebodyeats.com/wp-content/uploads/2021/11/thanksgiving-cheese-board.jpg", "conf": 0.1, "detect_objects": "{\n  \"bowl\": 5,\n  \"cup\": 2,\n  \"sandwich\": 1,\n  \"dining table\": 1,\n  \"knife\": 1,\n  \"cake\": 1\n}"}
Assistant: The objects detected in the image with a confidence threshold of 0.1 are as follows:

- Bowl: 5
- Cup: 2
- Sandwich: 1
- Dining table: 1
- Knife: 1
- Cake: 1

If you need any further assistance, feel free to ask!
